# Final Report (Part 1)
## Week 4 Assignment

# 1. Introduction:

Clearly define a problem and the audience who would be interested in this project.

### 1.1 Scenario and Background

There are more and more international students choose to study aboard these days. More than 100,000 students from around the world came to study in New York in 2017, which is 11% of all international students coming to the United States. New York is undoubtedly one of the most student-preferred cities in the country, and for good reason. However, when those students first come to New York, find a good neighborhood to live sometimes can be stressful. It also takes a lot of time for international students to search unstructured information from multiple sources such as internet, newsletter, google etc.

Findinng a good neighborhood to live is easy. There are so many factors need to be taken into consideration:
* Transportation - students would more prefere to choose place with comvenient transportation.
* Ammenities in the neighborhood - such as international cousine restaurants, cafes, food shops and entertainment. 

For this final project, I would like to apply the data science skills I learned from the Coursera course to give those international students more insights and information when choosing neighborhood in Manhattan.


### 1.2 Description of the Problem

The problem I'm trying to resolve is tp help Lam - an internationl student to find a rental apartment unit in Manhattan, New York to meet his requirement best based on their preference. For this case, Lam wants his new rental unit meet the following conditions:
* The rental unit located in Manhattan, New York
* The rental unit has at least 2 bedrooms
* The monthly rental not to exceed US$7500/month
* The rental unit located in a radius of 1 mile of a subway station
* The rental unit located in the neighborhood with ammenities and venues similar to Lam's home in Hongkong. ( Lam currently resides in the neighborhood of Tsim Sha Tsui East in Honkong. Based on venues around the area of residence data provided by FourSquare - which are shown in the Hongkong map shown in methodology and execution in section 3.0. It serves as a reference for comparison with the desired future location in Manhattan, New York.

### 1.3 Potential Audience 

The data science methodology I implement in this case will be beneficial to anyone who want to find a ideal neighborhood/house/apartment in foreign country. You may just recieved a job offer from your dream company in another city/country you are not familiar with but you need to relocate immediately; You may just recieved the offer from your dream college and need to find a good neighborhood to live... My approach and meghodologies will help you to solve this kind of problems.

# 2. Data

Describe the data that you will be using to solve the problem or execute your idea. 

### 2.1 Manhattan Neighborhoods Data

In order to help Lam find a rental unit in Manhattan with similar conditions as his apartment in Hongkong, I retrieve a <b>the Manhattan Neighborhoods Data</b> from Google. This dataset was produced with FourSquare during course lab work 3-3-2. The dataset has 4 columns:

* Borough	
* Neighborhood	
* Latitude	
* Longitude	
* Cluster Labels

In [3]:
manhattan_data = pd.read_csv('https://raw.githubusercontent.com/CloudyGreen/Coursera_Capstone/master/mh_neigh_data.csv')
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels
0,Manhattan,Marble Hill,40.876551,-73.910660,2
1,Manhattan,Chinatown,40.715618,-73.994279,2
2,Manhattan,Washington Heights,40.851903,-73.936900,4
3,Manhattan,Inwood,40.867684,-73.921210,3
4,Manhattan,Hamilton Heights,40.823604,-73.949688,0


### 2.2 Manhattan Subway Stations Geographic Data

Since we need to know the transportation information near the rental unit. It's necessary for us to obtain a Manhattan Subway Station data. I get this data from <a href='https://en.wikipedia.org/wiki/List_of_New_York_City_Subway_stations_in_Manhattan'>Wikipedia Page</a>, and just converted it into csv. Also, we can get the information from NY Transit authority and <a href='(https://www.google.com/maps/search/manhattan+subway+metro+stations/@40.7837297,-74.1033043,11z/data=!3m1!4b1'>Google Maps</a>. Then I'm able to combine the list of subway stops names and their address. The geolocation (Latitude and Longitude) was obtained via an algorythm using Nominatim.

In [4]:
manhattan_subway=pd.read_csv('https://raw.githubusercontent.com/CloudyGreen/Coursera_Capstone/master/mh_subway.csv')
manhattan_subway.head()

,sub_station,sub_address,lat,long
0,Dyckman Street Subway Station,"170 Nagle Ave, New York, NY 10034, USA",40.861857,-73.924509
1,57 Street Subway Station,"New York, NY 10106, USA",40.764250,-73.954525
2,Broad St,"New York, NY 10005, USA",40.730862,-73.987156
3,175 Street Station,"807 W 177th St, New York, NY 10033, USA",40.847991,-73.939785
4,5 Av and 53 St,"New York, NY 10022, USA",40.764250,-73.954525


### 2.3 Manhattan Rental House Data

Besides, I retrieved a list of places for rent from a real estate companies called <a href='www.rentmanhattan.com'>Rent Manhattan</a>. I exported all of the data in a single csv file. It includes the following columns:

* Address: address of rental unit
* Area: areas of Manhattan
* Price_per_ft2: price per square feet
* Rooms: numbers of rooms
* Area_ft2: area square feet
* Lat: latitude
* Long: longtitude

With the use of `geolocator = Nominatim()`, we are able to get the latitude and longiude for the Manhattan subway stantion locations and for each rental unit listed.

In [5]:
manhattan_house=pd.read_csv('https://raw.githubusercontent.com/CloudyGreen/Coursera_Capstone/master/mh_rent_latlong.csv')
manhattan_house.head()

,Address,Area,Price_per_ft2,Rooms,Area-ft2,Rent_Price,Lat,Long
0,West 105th Street,Upper West Side,2.94,5.0,3400,10000,40.799771,-73.966213
1,East 97th Street,Upper East Side,3.57,3.0,2100,7500,40.788585,-73.955277
2,West 105th Street,Upper West Side,1.89,4.0,2800,5300,40.799771,-73.966213
3,CARMINE ST.,West Village,3.03,2.0,1650,5000,40.730523,-74.001873
4,171 W 23RD ST.,Chelsea,3.45,2.0,1450,5000,40.744118,-73.995299


In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    ca-certificates-2020.4.5.1 |       hecc5488_0         146 KB  conda-forge
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    certifi-2020.4.5.1         |   py36h9f0ad1d_0         151 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1